## Medical Image Segmentation - Standard U-NET implementation

In [1]:
%load_ext autoreload
%autoreload 2

#### Import modules

In [2]:
import sys
from pathlib import Path
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

2025-04-29 11:18:30.211422: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-29 11:18:30.217083: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-29 11:18:30.232839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745918310.257738   81899 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745918310.265122   81899 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745918310.283947   81899 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

#### Local imports

In [3]:
sys.path.append(str(Path(".").resolve().parent))

from scripts.experiments import InputOutput
io = InputOutput()

### Step 1 - Loading training data

In [4]:
dwi_sequence, dwi_segmentation = io.read_nrrds("../MRI_data/ROI/DWI")
sequence_header, sequence_vol = io.nrrd_to_matrix(dwi_sequence)
segmentation_header, segmentation_vol = io.nrrd_to_matrix(dwi_segmentation)

#### Pre-processing data
`[TO-DO] - Build the volume fixer into the InputOutput module`

In [5]:
def fix_volume_shape(volume):
    """
    This module fixes the shape of a volume if there is a transposed image.
    Sometimes this happens for the DWI sequence.
    """
    if volume.shape[0] == 128 and volume.shape[1] == 104:
        volume = np.transpose(volume, (1, 0, 2))
    return volume

sequence_vol = [fix_volume_shape(vol) for vol in sequence_vol]
segmentation_vol = [fix_volume_shape(vol) for vol in segmentation_vol]

#### Save image volumes to an array of images

In [6]:
sequence_vol = np.concatenate(sequence_vol, axis=2)
segmentation_vol = np.concatenate(segmentation_vol, axis=2)
sequence_vol.shape, segmentation_vol.shape

((104, 128, 570), (104, 128, 570))

#### Image normalization
Succesfully training u-nets requires the image data to be standardized. The raw sequences produced by the MRI scanner contain extreme values which do not make sense to preserve in the scope of object detection.  

In [7]:
sequence_vol_n = sequence_vol / sequence_vol.max()
sequence_vol.min(), sequence_vol.max(), sequence_vol_n.min(), sequence_vol_n.max()

(np.uint16(0), np.uint16(3517), np.float64(0.0), np.float64(1.0))

#### Sanity check for segmentation  
Our segmentation volume is supposed to have only two classes; either renal cortex `1` or not renal-cortex `0`. The code snippet below simply returns all unique values, this confirms there are only two values with a maximum of `1`.

In [8]:
np.unique(segmentation_vol)

array([0, 1], dtype=uint8)

### Step 2 - Declaring U-net